In [1]:
import cv2
import numpy as np
import dlib
import math
import time

cap = cv2.VideoCapture(0) # Start the camera

detector = dlib.get_frontal_face_detector() # Face Detection
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat") # Face landmarks detection
font = cv2.FONT_HERSHEY_SIMPLEX

faceProto = "models/opencv_face_detector.pbtxt"
faceModel = "models/opencv_face_detector_uint8.pb"
ageProto = "models/age_deploy.prototxt"
ageModel = "models/age_net.caffemodel"
genderProto = "models/gender_deploy.prototxt"
genderModel = "models/gender_net.caffemodel"
# MODEL AVERAGE
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
ageList = [1, 5, 10, 16, 28, 40, 50, 70]
genderList = ['Male', 'Female']
# LOAD THE DNN
ageNet = cv2.dnn.readNet(ageModel, ageProto)
genderNet = cv2.dnn.readNet(genderModel, genderProto)
faceNet = cv2.dnn.readNet(faceModel, faceProto)

class User: # Define a user class
    def __init__(self, name, gender, age, engage, speaking):
        self.name = name
        self.gender = gender
        self.age = age
        self.engage = engage
        self.speaking = speaking
        self.count = 0

    def describe(self):
        print("Name: "+self.name)
        print("Gender: "+self.gender)
        print("Age: "+self.age)
        print("Engage: "+self.engage)
        print("Speaking: "+self.speaking)

    def updateGender(self, gender):
        self.gender = gender
        print("Gender is updated to: "+ self.gender)
    def updateAge(self, age):
        self.age = age
        print("Age is updated to: "+ str(self.age))
    def updateEngage(self,engage):
        self.engage = engage
        if self.engage:
            print("Engaged")
        else:
            print("Not engaged!")
    def updateSpeaking(self,speaking):
        self.speaking = speaking
        if self.speaking:
            print("Speaking")
        else:
            print("Not speaking!")

def midpoint(p1,p2): # Get middle point from 2 points
    return int((p1.x+p2.x)/2),int((p1.y+p2.y)/2)

def get_blinking_ratio(eye_points, facial_landmarks): # Eye Detection, Engage Detection
    eye_left = (facial_landmarks.part(eye_points[0]).x,facial_landmarks.part(eye_points[0]).y)
    eye_right = (facial_landmarks.part(eye_points[3]).x,facial_landmarks.part(eye_points[3]).y)
    eye_top = midpoint(facial_landmarks.part(eye_points[1]),facial_landmarks.part(eye_points[2]))
    eye_bottom = midpoint(facial_landmarks.part(eye_points[5]),facial_landmarks.part(eye_points[4]))

    hor_line = cv2.line(frame,eye_left,eye_right,(0,255,0),2)
    ver_line = cv2.line(frame,eye_top,eye_bottom,(0,255,0),2)

    ver_line_length = math.hypot((eye_top[0] - eye_bottom[0]),(eye_top[1] - eye_bottom[1]))
    hor_line_length = math.hypot((eye_left[0] - eye_right[0]),(eye_left[1]-eye_right[1]))

    eye_ratio = hor_line_length/ver_line_length
    return eye_ratio

def get_speaking_ratio(facial_landmarks):
    mouth_top = (facial_landmarks.part(62).x,facial_landmarks.part(62).y)
    mouth_bottom = (facial_landmarks.part(66).x,facial_landmarks.part(66).y)
    mouth_left = (facial_landmarks.part(48).x,facial_landmarks.part(48).y)
    mouth_right = (facial_landmarks.part(54).x,facial_landmarks.part(54).y)
    hor_line = cv2.line(frame, mouth_left,mouth_right,(0,255,0),2)
    ver_line = cv2.line(frame, mouth_top,mouth_bottom,(0,255,0),2)

    ver_line_length = math.hypot((mouth_top[0] - mouth_bottom[0]),(mouth_top[1] - mouth_bottom[1]))
    hor_line_length = math.hypot((mouth_left[0] - mouth_right[0]),(mouth_left[1]-mouth_right[1]))

    if ver_line_length != 0:
        mouth_ratio = hor_line_length / ver_line_length
    else:
        mouth_ratio = 0
    return mouth_ratio

def outputLabel(user): #Modify Output Label On Frames
    if user.engage:
        engage = 'Engaged'
    else:
        engage = 'NOT Engaged'
    if user.speaking:
        speaking = 'Speaking'
    else:
        speaking = 'NOT Speaking'
    if user.age < 18:
        user_type = 'Child'
    else:
        user_type = 'Adult'
    label = "{},{},Age: {},{},{},{}".format(user.name, user.gender,user.age,user_type ,engage,speaking)
    return label

padding = 20
while True:
    _, frame = cap.read() # Read the camera
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # Trans to grayscale

    faces = detector(gray) # Face detection

    for face in faces:
        user = User('User','Male',24,True,False)

        x1, y1 = face.left(),face.top()
        x2, y2 = face.right(),face.bottom()
        cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),2)

        # Eye Detection, Engage Detection
        landmarks = predictor(gray, face)
        left_eye_ratio = get_blinking_ratio([36,37,38,39,40,41],landmarks)
        right_eye_ratio = get_blinking_ratio([42,42,44,45,46,47],landmarks)
        eye_ratio = (right_eye_ratio + left_eye_ratio) / 2
        if  eye_ratio > 5.6:
            user.updateEngage(False)

        # Age and Gender Detection
        face_cnn = frame[max(0, y1 - padding):min(y2 + padding, frame.shape[0] - 1),
               max(0, x1 - padding):min(x2 + padding, frame.shape[1] - 1)]

        blob = cv2.dnn.blobFromImage(face_cnn, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)

        genderNet.setInput(blob)
        genderPreds = genderNet.forward()
        gender = genderList[genderPreds[0].argmax()]
        user.updateGender(gender)

        ageNet.setInput(blob)
        agePreds = ageNet.forward()
        age = ageList[agePreds[0].argmax()]
        user.updateAge(age)

        # Speaking Detection
        mouth_ratio = get_speaking_ratio(landmarks)
        if mouth_ratio < 22:
            user.updateSpeaking(True)
        else:
            user.updateSpeaking(False)

        # TODO: Objects Detection


        label = outputLabel(user)
        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,0), 2, cv2.LINE_AA)

    if cv2.waitKey(1) == ord('q'):# Esc
        break

    cv2.imshow("ROBOT IS WATCHING YOU",frame)

cap.release()
cv2.destroyAllWindows()


Gender is updated to: Male
Age is updated to: 28
Not speaking!
Gender is updated to: Male
Age is updated to: 28
Speaking
Not engaged!
Gender is updated to: Male
Age is updated to: 28
Not speaking!
Gender is updated to: Male
Age is updated to: 28
Not speaking!
Gender is updated to: Male
Age is updated to: 28
Not speaking!
Gender is updated to: Male
Age is updated to: 28
Not speaking!
Gender is updated to: Male
Age is updated to: 28
Not speaking!
Gender is updated to: Male
Age is updated to: 28
Not speaking!
Gender is updated to: Male
Age is updated to: 28
Not speaking!
Gender is updated to: Male
Age is updated to: 28
Not speaking!
Gender is updated to: Male
Age is updated to: 28
Not speaking!
Gender is updated to: Female
Age is updated to: 28
Not speaking!
Gender is updated to: Male
Age is updated to: 28
Not speaking!
Gender is updated to: Male
Age is updated to: 28
Not speaking!
Gender is updated to: Male
Age is updated to: 28
Not speaking!
Gender is updated to: Male
Age is updated to: